In [1]:
import sqlite3 as sql
import utils.sql_utils as u
from pathlib import Path
import pandas as pd

%load_ext autoreload
%autoreload 2

# Pokemon DB

In [2]:
POKEMON_DB = Path(r"resources\pokemon.db")

def query(q: str, connection: sql.Connection=None) -> pd.DataFrame:
    return u.query(q, conn=connection, db_path=POKEMON_DB, as_dataframe=True)


## Create and populate tables from SQL files

### Create Tables

In [ ]:
u.initialize_database(POKEMON_DB, [Path("resources/sql_scripts/pokemon_db/create_tables.sql")])

### Insert Records

In [46]:
sql_paths = list(Path("resources/sql_scripts/pokemon_db").glob("insert_*.sql"))

u.initialize_database(POKEMON_DB, sql_paths)

resources\sql_scripts\pokemon_db\nature_data.sql


### Data Quality Checks

In [ ]:
# Confirm that the trainers_pokemon are all in the pokedex
# note that this wasn't the case before and I fixed that in the sql files

q = """
SELECT t.*
FROM trainers as t
where t.pokemon NOT IN (
    SELECT pok_name from pokemon
)
"""

query(q)

### Fix Issues

In [ ]:
# Some cleanup to align names between sources
with sql.connect(POKEMON_DB) as conn:
    cur = conn.cursor()
    cur.execute(f"UPDATE trainers SET pokemon = LOWER(pokemon);")
    cur.execute(f"UPDATE pokemon SET pok_name = 'basculin' WHERE pok_name = 'basculin-red-striped';")
    cur.execute(f"UPDATE trainers SET pokemon = 'nidoran-m' WHERE pokemon = 'nidoran♂';")
    conn.commit()

In [48]:
col_rename = {
    "b_hp": "hp",
    "b_atk": "attack",
    "b_def": "defense",
    "b_sp_atk": "spatk",
    "b_sp_def": "spdef",
    "b_speed": "speed",
}

with sql.connect(POKEMON_DB) as conn:
    cur = conn.cursor()
    for old, new in col_rename.items():
        cur.execute(f"ALTER TABLE `base_stats` RENAME COLUMN {old} TO {new};")
    conn.commit()

## Queries

In [ ]:
u.list_table_schemas(POKEMON_DB) 
# Note - Saved as resources/images/pokemonDB_schema.PNG ... so open in side window

In [ ]:
# pokemon ids for trainers pokemon

query("""
SELECT t.*, pk.pok_id
FROM trainers as t
JOIN pokemon as pk on pk.pok_name = t.pokemon
LIMIT 10
""")

In [ ]:
# try to figure out how the pokedex stats relate to trainers pokemon stats

query("""
SELECT t.*, b.*
FROM trainers as t
JOIN pokemon as pk on pk.pok_name = t.pokemon
JOIN base_stats as b on b.pok_id = pk.pok_id
ORDER BY b.hp + b.defense DESC
LIMIT 20
""")

# IVs, EVs, and Natures make this hard to figure out

In [56]:
# safe to assume full EVs for level 100 pokemon?
query("""
    SELECT t.*, b.*
    FROM trainers AS t
    JOIN pokemon AS pk ON pk.pok_name = t.pokemon
    JOIN base_stats AS b ON b.pok_id = pk.pok_id
    WHERE t.level = 100
""")

# Base stats are the average of the highest possible stats (beneficial nature, max EV and Max IV) and it's lowest possible stats (hindering nature, no EVs or IVs) on a level 100 pokemon.

,trainer,pokemon,level,hp,attack,defense,spatk,spdef,speed,pok_id,hp,attack,defense,spatk,spdef,speed
0,Cool♀,bulbasaur,100,236,139,139,171,171,131,1,45,49,49,65,65,45
1,Cool♀,bulbasaur,100,236,139,139,171,171,131,1,45,49,49,65,65,45
2,Cool♀,bulbasaur,100,236,139,139,171,171,131,1,45,49,49,65,65,45
3,Cool♀,bulbasaur,100,236,139,139,171,171,131,1,45,49,49,65,65,45
4,Cool♀,ivysaur,100,266,165,167,201,201,161,2,60,62,63,80,80,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,Bodybuilder♀ Powit,jirachi,100,346,241,241,241,241,241,385,100,100,100,100,100,100
912,Cooltrainer♀ Moden,jirachi,100,346,241,241,241,241,241,385,100,100,100,100,100,100
913,Cooltrainer♀ Moden,jirachi,100,346,241,241,241,241,241,385,100,100,100,100,100,100
914,Rider♀ Pulink,jirachi,100,346,241,241,241,241,241,385,100,100,100,100,100,100


In [62]:
# 252 / 4 = 63  EVs
# 32 IVs
# nature += 10%

# 100 = ((1.1*X + 63 + 32) + (X*0.9)) / 2

import math
def hp_of(base: int, IVs: int, EVs: int, lvl: int) -> int:
    return math.floor(0.01 * (2 * base + IVs + math.floor(0.25 * EVs)) * lvl) + lvl + 10

def stat_of(base: int, IVs: int, EVs: int, lvl: int, nature: float=1.0) -> int:
    return (math.floor(0.01 * (2 * base + IVs + math.floor(0.25 * EVs)) * lvl) + 5) * nature
# (floor(0.01 x (2 x Base + IV + floor(0.25 x EV)) x Level) + 5) x Nature.

# HP = floor(0.01 x (2 x Base + IV + floor(0.25 x EV)) x Level) + Level + 10
# Other Stats = (floor(0.01 x (2 x Base + IV + floor(0.25 x EV)) x Level) + 5) x Nature.

hp_of(100, 32, 63, 100), stat_of(100, 16, 0, 100, 1.1)

(357, 243.10000000000002)